# 4 - Saccade Identification

In [18]:
%matplotlib qt
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [21, 8]
plt.rcParams['figure.dpi'] = 100

In [19]:
from bsp.core import load_study, calibration, differentiate

study = load_study("data/prueba.bsp")
test = study[1]

In [20]:
test.annotate()

In [21]:
plt.plot(test.hor_channel)

In [15]:
study.hor_calibration

0.002237178257431945

In [17]:
test._hor_channel

array([  0.      ,   0.      ,   0.      , ..., -14.037979, -14.02903 ,
       -14.002186], dtype=float32)